<a href="https://colab.research.google.com/github/vialucis/BreastCancerML/blob/master/IT3011_project_non_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Breast Cancer Using Image Classification

In [0]:
import os
import math

import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from skimage.io import imread, imshow
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics

import cv2
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

PROJ_ROOT = "/content/drive/My Drive/Dataset(reshaped)/200X"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#proposal for dataloader

# define transformation that you wish to apply on image
data_transforms = transforms.Compose([
                    transforms.CenterCrop(224),
                    transforms.ToTensor()])
# Load the datasets with ImageFolder
image_datasets = datasets.ImageFolder(root= PROJ_ROOT, transform=data_transforms)
image_datasets_grey = datasets.ImageFolder(root= PROJ_ROOT, transform = transforms.Compose([transforms.CenterCrop(224),transforms.Grayscale(num_output_channels=1),transforms.ToTensor()]))
image_datasets_PIL = datasets.ImageFolder(root= PROJ_ROOT, transform = transforms.Compose([transforms.CenterCrop(224)]))

n = len(image_datasets)  # total number of examples
test_percent = 0.2

n_test = int(test_percent * n)  # take ~10% for test
test_set = torch.utils.data.Subset(image_datasets, range(n_test))  # take first 10%
train_set = torch.utils.data.Subset(image_datasets, range(n_test, n))  # take the rest

# Using the image datasets and the transforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_set.dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set.dataset, batch_size=32, shuffle=True, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available() == True:
  print('GPU available')

2020
404
1616
GPU available


In [0]:
x = [i[0] for i in image_datasets]
y = image_datasets.targets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 100

## Support Vector Machine

### SVM - Mean pixel

In [0]:
"""
Instead of using the pixel values from the three channels separately, 
we can generate a new matrix that has the mean value of pixels from all three channels.
"""
# input: array length 1616 or 404 with each element 3x224x224

def mean_pixel (dataset):
  # create main matrix 
  mean_pixel_matrix = np.zeros((1,224*224)) 

  # calculate the mean pixel values of channels 
  for idx in range(len(dataset)):
    feature_matrix = np.zeros((224,224)) 
    for i in range(224):
        for j in range(224):
            feature_matrix[i][j] = ((dataset[idx][0][i][j]) + int(dataset[idx][1][i][j]) + int(dataset[idx][2][i][j]))/3

    # reshape 2D -> 1D
    features = np.reshape(feature_matrix, (224*224)) 
    features.shape

    # add to main matrix
    mean_pixel_matrix = np.vstack((mean_pixel_matrix, features))

    # update status every 10 elements
    if idx % 10 == 0:
      print(mean_pixel_matrix.shape)

  mean_pixel_matrix = np.delete(mean_pixel_matrix, (0), axis=0)

  return (mean_pixel_matrix)

In [0]:
mean_pixel_train = mean_pixel(x_train)
mean_pixel_test = mean_pixel(x_test)

for kernel in ('linear', 'poly', 'rbf'):
  clf = svm.SVC(kernel=kernel)
  clf.fit(mean_pixel_train,y_train)
  y_pred = clf.predict(mean_pixel_test)
  print("mean pixel accuracy: ", accuracy_score(y_test,y_pred))
  print(confusion_matrix(y_test,y_pred))
  print(classification_report(y_test,y_pred))

(2, 50176)
(12, 50176)
(22, 50176)
(32, 50176)
(42, 50176)
(52, 50176)
(62, 50176)
(72, 50176)
(82, 50176)
(92, 50176)
(102, 50176)
(112, 50176)
(122, 50176)
(132, 50176)
(142, 50176)
(152, 50176)
(162, 50176)
(172, 50176)
(182, 50176)
(192, 50176)
(202, 50176)
(212, 50176)
(222, 50176)
(232, 50176)
(242, 50176)
(252, 50176)
(262, 50176)
(272, 50176)
(282, 50176)
(292, 50176)
(302, 50176)
(312, 50176)
(322, 50176)
(332, 50176)
(342, 50176)
(352, 50176)
(362, 50176)
(372, 50176)
(382, 50176)
(392, 50176)
(402, 50176)
(412, 50176)
(422, 50176)
(432, 50176)
(442, 50176)
(452, 50176)
(462, 50176)
(472, 50176)
(482, 50176)
(492, 50176)
(502, 50176)
(512, 50176)
(522, 50176)
(532, 50176)
(542, 50176)
(552, 50176)
(562, 50176)
(572, 50176)
(582, 50176)
(592, 50176)
(602, 50176)
(612, 50176)
(622, 50176)
(632, 50176)
(642, 50176)
(652, 50176)
(662, 50176)
(672, 50176)
(682, 50176)
(692, 50176)
(702, 50176)
(712, 50176)
(722, 50176)
(732, 50176)
(742, 50176)
(752, 50176)
(762, 50176)
(772, 5017

### SVM - Filters

In [0]:
from PIL import Image, ImageFilter

loader_to_tensor = transforms.Compose([transforms.ToTensor()])

def filters(image):
  image_findedges = image.filter(ImageFilter.FIND_EDGES)
  image_contour = image.filter(ImageFilter.CONTOUR)
  image_edgeenhance = image.filter(ImageFilter.EDGE_ENHANCE)
  image_emboss = image.filter(ImageFilter.EMBOSS)

  return(image, image_findedges, image_contour, image_edgeenhance, image_emboss)

def PIL_to_tensor(filtered_images):
  image = loader_to_tensor(filtered_images[0])
  image_findedges = loader_to_tensor(filtered_images[1])
  image_contour = loader_to_tensor(filtered_images[2])
  image_edgeenhance = loader_to_tensor(filtered_images[3])
  image_emboss = loader_to_tensor(filtered_images[4])
  return(image, image_findedges, image_contour, image_edgeenhance, image_emboss)

x_filter = np.array([])
x_filter_findedges = np.array([])
x_filter_contour = np.array([])
x_filter_edgeenhance = np.array([])
x_filter_emboss = np.array([])

# filter whole dataset
for idx in image_datasets_PIL:
  image = idx[0]
  # filter images in PIL form
  filtered_images = filters(image)
  # transform PIL to tensors
  filtered_images_tensor = PIL_to_tensor(filtered_images) # shape: 5 x torch.Size([3, 224, 224])
  # create 9 lists of tensors
  x_filter = np.append(x_filter, filtered_images_tensor[0])
  x_filter_findedges = np.append(x_filter_findedges, filtered_images_tensor[1])
  x_filter_contour = np.append(x_filter_contour, filtered_images_tensor[2])
  x_filter_edgeenhance = np.append(x_filter_edgeenhance, filtered_images_tensor[3])
  x_filter_emboss = np.append(x_filter_emboss, filtered_images_tensor[4])

2020
3.0 3.0 3.0
6.0 6.0 6.0
9.0 9.0 9.0
12.0 12.0 12.0
15.0 15.0 15.0
18.0 18.0 18.0
21.0 21.0 21.0
24.0 24.0 24.0
27.0 27.0 27.0
30.0 30.0 30.0
33.0 33.0 33.0
36.0 36.0 36.0
39.0 39.0 39.0
42.0 42.0 42.0
45.0 45.0 45.0
48.0 48.0 48.0
51.0 51.0 51.0
54.0 54.0 54.0
57.0 57.0 57.0
60.0 60.0 60.0
63.0 63.0 63.0
66.0 66.0 66.0
69.0 69.0 69.0
72.0 72.0 72.0
75.0 75.0 75.0
78.0 78.0 78.0
81.0 81.0 81.0
84.0 84.0 84.0
87.0 87.0 87.0
90.0 90.0 90.0
93.0 93.0 93.0
96.0 96.0 96.0
99.0 99.0 99.0
102.0 102.0 102.0
105.0 105.0 105.0
108.0 108.0 108.0
111.0 111.0 111.0
114.0 114.0 114.0
117.0 117.0 117.0
120.0 120.0 120.0
123.0 123.0 123.0
126.0 126.0 126.0
129.0 129.0 129.0
132.0 132.0 132.0
135.0 135.0 135.0
138.0 138.0 138.0
141.0 141.0 141.0
144.0 144.0 144.0
147.0 147.0 147.0
150.0 150.0 150.0
153.0 153.0 153.0
156.0 156.0 156.0
159.0 159.0 159.0
162.0 162.0 162.0
165.0 165.0 165.0
168.0 168.0 168.0
171.0 171.0 171.0
174.0 174.0 174.0
177.0 177.0 177.0
180.0 180.0 180.0
183.0 183.0 183.0
186.0

In [0]:
# split x into 2020 subarrays
n = len(image_datasets_PIL)
x_filter_split = np.split(x_filter, n)
x_filter_findedges_split = np.split(x_filter_findedges, n)
x_filter_contour_split = np.split(x_filter_contour, n)
x_filter_edgeenhance_split = np.split(x_filter_edgeenhance, n)
x_filter_emboss_split = np.split(x_filter_emboss, n)

In [0]:
y = image_datasets_PIL.targets 
x_train_f, x_test_f, y_train_f, y_test_f = train_test_split(x_filter_split, y, test_size=0.2, random_state=42)
x_train_f_findedges, x_test_f_findedges, y_train_f_findedges, y_test_f_findedges = train_test_split(x_filter_findedges_split, y, test_size=0.2, random_state=42)
x_train_f_contour, x_test_f_contour, y_train_f_contour, y_test_f_contour = train_test_split(x_filter_contour_split, y, test_size=0.2, random_state=42)
x_train_f_edgeenhance, x_test_f_edgeenhance, y_train_f_edgeenhance, y_test_f_edgeenhance = train_test_split(x_filter_edgeenhance_split, y, test_size=0.2, random_state=42)
x_train_f_emboss, x_test_f_emboss, y_train_f_emboss, y_test_f_emboss = train_test_split(x_filter_emboss_split, y, test_size=0.2, random_state=42)

x_train_f done
x_train_f_findedges done
x_train_f_contour done
x_train_f_edgeenhance done
x_train_f_emboss done
1616


In [0]:
def mean_pixel_1D (dataset):
  # create main matrix 
  mean_pixel_matrix = np.zeros((1,224*224)) 

  # calculate the mean pixel values of channels 
  for idx in range(len(dataset)):
    features = [] 
    for i in range(224*224):
      features.append((dataset[idx][i] + dataset[idx][i + 224*224] + dataset[idx][i + 2*224*224])/3)

    # add to main matrix
    mean_pixel_matrix = np.vstack((mean_pixel_matrix, features))

    # update status every 10 elements
    if idx % 10 == 0:
      print(mean_pixel_matrix.shape)

  mean_pixel_matrix = np.delete(mean_pixel_matrix, (0), axis=0)

  return (mean_pixel_matrix)

In [0]:
for x_train, x_test, y_train, y_test in zip([x_train_f_findedges, x_train_f_contour, x_train_f_edgeenhance, x_train_f_emboss], [x_test_f_findedges, x_test_f_contour, x_test_f_edgeenhance, x_test_f_emboss], [y_train_f_findedges, y_train_f_contour, y_train_f_edgeenhance, y_train_f_emboss], [y_test_f_findedges, y_test_f_contour, y_test_f_edgeenhance, y_test_f_emboss]):
  mean_pixel_train = mean_pixel_1D(x_train) #input: 1616 * 150528 
  mean_pixel_test = mean_pixel_1D(x_test)
  print("new filter:")
  for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel)
    clf.fit(mean_pixel_train,y_train)
    print('kernel: ', kernel)
    y_pred = clf.predict(mean_pixel_test)
    print("mean pixel accuracy: ", accuracy_score(y_test,y_pred))
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

(2, 50176)
(12, 50176)
(22, 50176)
(32, 50176)
(42, 50176)
(52, 50176)
(62, 50176)
(72, 50176)
(82, 50176)
(92, 50176)
(102, 50176)
(112, 50176)
(122, 50176)
(132, 50176)
(142, 50176)
(152, 50176)
(162, 50176)
(172, 50176)
(182, 50176)
(192, 50176)
(202, 50176)
(212, 50176)
(222, 50176)
(232, 50176)
(242, 50176)
(252, 50176)
(262, 50176)
(272, 50176)
(282, 50176)
(292, 50176)
(302, 50176)
(312, 50176)
(322, 50176)
(332, 50176)
(342, 50176)
(352, 50176)
(362, 50176)
(372, 50176)
(382, 50176)
(392, 50176)
(402, 50176)
(412, 50176)
(422, 50176)
(432, 50176)
(442, 50176)
(452, 50176)
(462, 50176)
(472, 50176)
(482, 50176)
(492, 50176)
(502, 50176)
(512, 50176)
(522, 50176)
(532, 50176)
(542, 50176)
(552, 50176)
(562, 50176)
(572, 50176)
(582, 50176)
(592, 50176)
(602, 50176)
(612, 50176)
(622, 50176)
(632, 50176)
(642, 50176)
(652, 50176)
(662, 50176)
(672, 50176)
(682, 50176)
(692, 50176)
(702, 50176)
(712, 50176)
(722, 50176)
(732, 50176)
(742, 50176)
(752, 50176)
(762, 50176)
(772, 5017

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kernel:  poly
mean pixel accuracy:  0.40594059405940597
[[  0   0   0   0  20   0   1   0]
 [  0   1   0   9  49   0   0   0]
 [  0   0   0   0  20   0   1   0]
 [  0   0   0   4  33   0   0   0]
 [  0   0   0   8 158   2   0   0]
 [  0   0   0   1  33   0   0   0]
 [  0   1   0   5  28   0   1   0]
 [  0   0   0   8  21   0   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.50      0.02      0.03        59
           2       0.00      0.00      0.00        21
           3       0.11      0.11      0.11        37
           4       0.44      0.94      0.60       168
           5       0.00      0.00      0.00        34
           6       0.33      0.03      0.05        35
           7       0.00      0.00      0.00        29

    accuracy                           0.41       404
   macro avg       0.17      0.14      0.10       404
weighted avg       0.29      0.41      0.27       404



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kernel:  rbf
mean pixel accuracy:  0.4158415841584158
[[  0   0   0   0  21   0   0   0]
 [  0   0   0   0  59   0   0   0]
 [  0   0   0   0  21   0   0   0]
 [  0   0   0   0  37   0   0   0]
 [  0   0   0   0 168   0   0   0]
 [  0   0   0   0  34   0   0   0]
 [  0   0   0   0  35   0   0   0]
 [  0   0   0   0  29   0   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.00      0.00      0.00        59
           2       0.00      0.00      0.00        21
           3       0.00      0.00      0.00        37
           4       0.42      1.00      0.59       168
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00        29

    accuracy                           0.42       404
   macro avg       0.05      0.12      0.07       404
weighted avg       0.17      0.42      0.24       404

(2, 50176)


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kernel:  poly
mean pixel accuracy:  0.42574257425742573
[[  0   1   0   1  18   0   1   0]
 [  0   3   0   7  44   1   3   1]
 [  0   1   0   0  17   0   3   0]
 [  0   6   0   6  24   0   1   0]
 [  0   2   0   4 157   5   0   0]
 [  0   1   0   0  32   1   0   0]
 [  0   3   0   3  25   0   4   0]
 [  0   1   0   7  20   0   0   1]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.17      0.05      0.08        59
           2       0.00      0.00      0.00        21
           3       0.21      0.16      0.18        37
           4       0.47      0.93      0.62       168
           5       0.14      0.03      0.05        34
           6       0.33      0.11      0.17        35
           7       0.50      0.03      0.06        29

    accuracy                           0.43       404
   macro avg       0.23      0.17      0.15       404
weighted avg       0.31      0.43      0.31       404



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kernel:  rbf
mean pixel accuracy:  0.41336633663366334
[[  0   0   0   0  21   0   0   0]
 [  0   3   0   0  56   0   0   0]
 [  0   0   0   0  21   0   0   0]
 [  0   2   0   0  35   0   0   0]
 [  0   3   0   0 164   1   0   0]
 [  0   0   0   0  34   0   0   0]
 [  0   2   0   0  33   0   0   0]
 [  0   1   0   0  28   0   0   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.27      0.05      0.09        59
           2       0.00      0.00      0.00        21
           3       0.00      0.00      0.00        37
           4       0.42      0.98      0.59       168
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00        29

    accuracy                           0.41       404
   macro avg       0.09      0.13      0.08       404
weighted avg       0.21      0.41      0.26       404



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM - All in one

In [0]:
"""
  Takes in a Dataset which is a list of Tensors and writes every entry into a long vector with num = the number of Images as entries.
  In the end we split the images into train and test to fit it into different ML-algorithms
"""
  
def AllinOne(dataset):

  # CUDA for PyTorch
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")
  #cudnn.benchmark = True

  loader = DataLoader(dataset, shuffle=True)

  g= np.array([])
  h= np.array([])
  n = len(dataset)

  for x, y in loader:
    g = np.append(g,x) # image
    h = np.append(h,y) # image label

    print(g.shape)

  g_split = np.split(g, n)

  x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(g_split, h, test_size=0.2, random_state=42)

  return x_train_g, x_test_g, y_train_g, y_test_g

In [0]:
x_train_g, x_test_g, y_train_g, y_test_g = AllinOne(image_datasets)

<class 'torchvision.datasets.folder.ImageFolder'>
['benign_adenosis', 'benign_fibroadenoma', 'benign_phyllodes_tumor', 'benign_tubular_adenoma', 'malignant_ductal_carcinoma', 'malignant_lobular_carcinoma', 'malignant_mucinous_carcinoma', 'malignant_papillary_carcinoma']
(150528,)
(301056,)
(451584,)
(602112,)
(752640,)
(903168,)
(1053696,)
(1204224,)
(1354752,)
(1505280,)
(1655808,)
(1806336,)
(1956864,)
(2107392,)
(2257920,)
(2408448,)
(2558976,)
(2709504,)
(2860032,)
(3010560,)
(3161088,)
(3311616,)
(3462144,)
(3612672,)
(3763200,)
(3913728,)
(4064256,)
(4214784,)
(4365312,)
(4515840,)
(4666368,)
(4816896,)
(4967424,)
(5117952,)
(5268480,)
(5419008,)
(5569536,)
(5720064,)
(5870592,)
(6021120,)
(6171648,)
(6322176,)
(6472704,)
(6623232,)
(6773760,)
(6924288,)
(7074816,)
(7225344,)
(7375872,)
(7526400,)
(7676928,)
(7827456,)
(7977984,)
(8128512,)
(8279040,)
(8429568,)
(8580096,)
(8730624,)
(8881152,)
(9031680,)
(9182208,)
(9332736,)
(9483264,)
(9633792,)
(9784320,)
(9934848,)
(10085376

KeyboardInterrupt: ignored

In [0]:
for kernel in ('linear', 'poly', 'rbf'):
  clf = svm.SVC(kernel=kernel)
  clf.fit(x_train_g,y_train_g)
  y_pred = clf.predict(x_test_g)
  print("grey method: ", accuracy_score(y_test_g,y_pred))
  #print(confusion_matrix(y_test_g,y_pred))
  print(classification_report(y_test_g,y_pred))

In [0]:
#logisticRegression

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train_g, y_train_g)
predictions = logisticRegr.predict(x_test_g)
# Use score method to get accuracy of model
score = logisticRegr.score(x_test_g, y_test_g)
print("Accuracy:", score)

cm = metrics.confusion_matrix(y_test_g, predictions)
print(cm)

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
#sns.set()

knn = KNeighborsClassifier(n_neighbors=150, metric='euclidean')
knn.fit(x_train_g, y_train_g)
y_pred = knn.predict(x_test_g)
score = knn.score(x_test_g, y_test_g)
print("Accuracy:",score)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics


cm = metrics.confusion_matrix(y_test_g, y_pred)
#print(cm)
print(classification_report(y_test_g,y_pred))

In [0]:
#SGD
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf.fit(x_train_g, y_train_g)
predictions = clf.predict(x_test_g)
# Use score method to get accuracy of model
score = clf.score(x_test_g, y_test_g)
print("Accuracy:",score)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

cm = metrics.confusion_matrix(y_test_g, predictions)
#print(cm)
print(classification_report(y_test_g,predictions))